In [ ]:
# Import the libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout 
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam, SGD

from sklearn.metrics import classification_report,confusion_matrix
from skimage.transform import resize

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier

import tensorflow as tf
import cv2



In [ ]:
############################################################
# PARAMETER SETTING
cnn_flag = True # train own CNN 

transfer_MobileNetV2_flag_1 = True 
# fine-tune pretrained MobileNetV2 

transfer_MobileNetV2_flag_2 = True 
# extract features with MobileNetV2 + classifiers

r = 56 # target image size        
ne = 2 ###########200 # number of epochs for CNN training
eta = 0.0001 # training rate

Videos = ['Pigs_49651_960_540_500f','Koi_5652_952_540',\
          'Pigeons_8234_1280_720','Pigeons_4927_960_540_600f', \
          'Pigeons_29033_960_540_300f']

cl_names = ['Linear Discriminant Analysis', '3-nn', 'Decision Tree',
            'SVM', 'Bagging', 'Random Forest']
###########################################

In [ ]:
##############################################
# FUNCTIONS
#---------------------------------------------------------
def convert_to_binary(x,n):
    u = list(np.unique(x))
    y = np.zeros((len(x),n))
    for i in range(len(x)):
        y[i,u.index(x[i])] = 1
    return y,u

#---------------------------------------------------------
def random_image_montage(imStore, grid_size = 4,):
    N = len(imStore)
    plt.figure(figsize = (10,10))
    
    T = grid_size**2
    if N < T: # not enough images for a full grid
        grid_size = np.floor(np.sqrt(N))
    
    # Choose random images from the collection
    chosen = np.random.permutation(N)
    chosen = chosen[:T]
    
    k = 1 # subplot index
    for i in range(T):
        img = imStore[chosen[i]]
                
        #resized_img = resize(img, (r, r))

        plt.subplot(grid_size,grid_size,k)
        plt.imshow(img)
        plt.axis('Off')
        k +=1   
        
#---------------------------------------------------------
def select_half_video(folder,bb_file,part = 1):
    
    bb = pd.read_csv(bb_file,header = None)    
    bb = bb.to_numpy()
   
    # Find the number of frames in the video
    lasf_file_name = bb[-1,5]
    f = np.int(lasf_file_name[5:10]) 
    half_f = np.floor(f/2).astype(int)
    
    images, labels = [],[]
    for i in range(len(bb)):
        z = bb[i,5] # take the file name
        nn = np.int(z[5:10]) # convert to numeric
        flag = ((part == 1) & (nn <= half_f)) | \
            ((part == 2) & (nn > half_f)) 
        if flag:
            label = bb[i,0]# take the string label
            label = label.replace(' ','') # trim the blanks
            filename = bb[i,5] 
            fn = label+'/'+ label + '_frame_' + filename[5:10] + '.jpg'
            img = plt.imread(folder+'/'+fn)
            if img is not None:
                images.append(img)
                labels.append(label)
            else:
                print(folder+'/'+fn)     
            
    return images, labels
        

In [ ]:
##############################################
# CALCULATIONS

for video in Videos:
    
    print('\n\n#####  ' + video + '  #####\n\n')
    
    folder = "../" + video + "_clips"
    bb_file = "../BB_" + video + ".csv"

    # Read training and testing data (half video; must not be random!)
    imds_1,labels_1 = select_half_video(folder,bb_file,part = 1) # training
    imds_2,labels_2 = select_half_video(folder,bb_file,part = 2) # testing
    
    # Repair the data (missing classes in training data)
    # Find missing classes and remove those from both sets

    ll = list(np.sort(np.unique(labels_1)))
    n_classes = len(ll)

    X_train_im = []
    X_train = []
    y_train_num = []
    for i in range(len(imds_1)):
        y_train_num.append(ll.index(labels_1[i]))
        X_train.append(resize(imds_1[i], (r, r)))
        X_train_im.append(imds_1[i])
    y_train,_ = convert_to_binary(y_train_num,n_classes)    
    X_train = np.array(X_train)/255                       

    X_test_im = []
    X_test = []
    y_test_num = [] # numerical testing labels
    for i in range(len(imds_2)):
        if labels_2[i] in set(ll):
            y_test_num.append(ll.index(labels_2[i]))
            X_test.append(resize(imds_2[i], (r, r)))
            X_test_im.append(imds_2[i])

    y_test,_ = convert_to_binary(y_test_num,n_classes)  
    X_test = np.array(X_test)/255    

    print('Training size = ',X_train.shape,' Testing size = ',X_test.shape)
    
    # Augment training data (just in case)
    datagen = ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            rotation_range = 30,  # randomly rotate images in the range (degrees, 0 to 180)
            zoom_range = 0.2, # Randomly zoom image 
            width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
            horizontal_flip = True,  # randomly flip images
            vertical_flip=False)  # randomly flip images

    datagen.fit(X_train)

    if cnn_flag:
        # CNN
        #==============================
        model = Sequential()
        model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(r,r,3)))
        model.add(Conv2D(64, (3, 3), activation='relu'))
        model.add(MaxPool2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(n_classes, activation='softmax'))
        #==============================

        opt = Adam(learning_rate = eta)
        model.compile(loss='binary_crossentropy',optimizer=opt, metrics=['accuracy'])
        history = model.fit(datagen.flow(X_train,y_train),epochs = ne, verbose = True)

        # without agumentation ---
        #history = model.fit(X_train,y_train,epochs = ne,verbose = False) 

        acc = history.history['accuracy']
        loss = history.history['loss']
        epochs_range = range(ne)
        plt.figure() # figsize=(15, 15))
        plt.subplot(121)
        plt.plot(epochs_range, acc)
        plt.title('Training Accuracy')
        plt.subplot(122)
        plt.plot(epochs_range, loss)
        plt.title('Training Loss')

        predictions = model.predict(X_test) # assigned probabilities
        y_assigned = np.argmax(predictions, axis=-1) # assigned labels

        testing_accuracy = np.mean(y_test_num == y_assigned)
        print('Testing accuracy = ', testing_accuracy)

        # Confusion matrix
        cm = confusion_matrix(y_test_num,y_assigned)
        plt.figure() 
        sns.heatmap(cm, xticklabels = ll, yticklabels = ll)        
        
        # Save results
        out_fn = video        

        TrainingAccLoss = np.hstack((np.reshape(acc,(-1,1)),np.reshape(loss,(-1,1))))
        ###########np.savetxt("TrainingAccLoss_"+out_fn+".csv", TrainingAccLoss, delimiter=",")
        
        TestingLabelsAssignedLabels = np.hstack( \
            (np.reshape(y_test_num,(-1,1)),np.reshape(y_assigned,(-1,1))))
        np.savetxt("TestingLabelsAssignedLabels_"+ \
            out_fn+".csv", TestingLabelsAssignedLabels, delimiter=",")

    if transfer_MobileNetV2_flag_1:
        #================================================================
        # TRANSFER LEARNING (1) MobileNetV2 - further NN model

        base_model = tf.keras.applications.MobileNetV2(input_shape = \
           (r, r, 3), include_top = False, weights = "imagenet")

        base_model.trainable = False
        model = tf.keras.Sequential([base_model,
             tf.keras.layers.GlobalAveragePooling2D(),
             tf.keras.layers.Dropout(0.2),
             tf.keras.layers.Dense(n_classes, activation="softmax")                                     
            ])
        base_learning_rate = eta
        model.compile(optimizer = \
                      tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
                      loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
                      metrics=['accuracy'])

        history = model.fit(datagen.flow(X_train, \
                     y_train),epochs = ne, verbose = True)

        predictions = model.predict(X_test) # assigned probabilities
        y_assigned = np.argmax(predictions, axis=-1) # assigned labels

        testing_accuracy = np.mean(y_test_num == y_assigned)
        print('Testing accuracy MobileNetV2= ', testing_accuracy)
        
        TestingLabelsAssignedLabels = np.hstack( \
            (np.reshape(y_test_num,(-1,1)),np.reshape(y_assigned,(-1,1))))
        ###########np.savetxt("TestingLabelsAssignedLabelsMobileNetV2_"+ \
        ###########    out_fn+".csv", TestingLabelsAssignedLabels, delimiter=",")
        
    if transfer_MobileNetV2_flag_2:        
        #================================================================
        # TRANSFER LEARNING (2) MobileNetV2 - classifiers
        base_model = tf.keras.applications.MobileNetV2(input_shape = \
                   (r, r, 3), include_top = False, weights = "imagenet")

        xx_train = base_model(X_train,training = False)
        xx_train_shaped = keras.layers.GlobalAveragePooling2D()(xx_train)

        xx_test = base_model(X_test,training = False)
        xx_test_shaped = keras.layers.GlobalAveragePooling2D()(xx_test)

        cla = []
        cla.append(LinearDiscriminantAnalysis())
        cla.append(KNeighborsClassifier())
        cla.append(DecisionTreeClassifier())
        cla.append(SVC(gamma=0.1, kernel="rbf"))
        cla.append(BaggingClassifier())
        cla.append(RandomForestClassifier())

        accs = np.zeros(len(cl_names))
        for i in range(len(cl_names)):
            classifier = cla[i]
            classifier.fit(xx_train_shaped,y_train_num)
            assigned_labels = classifier.predict(xx_test_shaped)
            accs[i] = np.mean(y_test_num == assigned_labels)
            print('%30s %.2f' % (cl_names[i], accs[i]))
            
        # Save results
        ###########np.savetxt("ClassifierAccMobileNetV2_"+ \
        ###########    out_fn+".csv", accs, delimiter=",")       
        
